### Some basic functions

In [9]:
import collections
import random
from math import *
import scipy.optimize as opt
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
import scipy
import random
import random as rng
from scipy.special import binom as binom_sp
import scipy.special as ss
from math import comb as binom
#def binom(n: int, k: int):
#    return binomial(n, k)
#    return scipy.special.comb(n, k)

#def binom(a, b):
#    return ss.binom(a, b)

def HHH(c):
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def H(c):
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    if k>n:
        return 0
    if k==n:
        return 0
    return n * HHH(k/n)

def binomHH(n,k):
    return n * HHH(k/n)

def time7diss(x):
    return -2*x/3+2/3

def time11diss(x):
    return -5/4*x+3/4

def float_range(start, stop, step):
    while start < stop:
        yield float(start)
        start += float(step)

def check_constraints(constraints, solution) : 
    return [ (constraint['type'], constraint['fun'](solution)) for constraint in constraints ]


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def round_to_str(t):
    """
    Rounds the value 't' to a string with 4 digit precision (adding trailing zeroes
    to emphasize precision).
    """
    s = str(round(t,4))
    # must be 6 digits
    return (s + "0" * (5 + s.find(".") -len(s)))
        

def round_upwards_to_str(t):
    """
    Rounds the value 't' *upwards* to a string with 4 digit precision (adding trailing zeroes
    to emphasize precision).
    """
    s = str( ceil(t*10000)/10000 )
    # must be 6 digits
    return (s + "0" * (5 + s.find(".") -len(s)))


def xlx(x):
    if x<=0: return 0
    return x*log(x, 2)


def p_good(a0, b0, a1, b1):
    return -2*xlx(a0/2) - 2*xlx(b0/2) - xlx(a1-a0/2) - xlx(b1-b0/2) - xlx(1-a1-b1-a0/2-b0/2) - 2*g(a1, b1)
    

def g(a, b):
    return -xlx(a) - xlx(b) - xlx(1-a-b)


def f(a, b, c):
    if a<=0: return g(b, c)
    if b<=0: return g(a, c)
    if c<=0: return g(a, b)
    if a+b+c >= 1: return min(g(b, c), g(a, c), g(a, b))
    try:
        return -a*log(a,2) - b*log(b,2) - c*log(c,2) - (1-a-b-c)*log(1-a-b-c, 2)
    except:
        return 0.

    
def p_good_2(b0, a0, c0, b1, a1, c1):
    def proba(x):
        return 2*xlx(a0/2) + 2*xlx(x+a1-a0/2-b0/2) + xlx(1-c0-2*a1-2*x) + 2*xlx(b0/2-x) + 2*xlx(x) + 2*xlx(x+c0/2-b1/2+a1/2-a0/4-b0/4) + xlx(b1-a1+a0/2+b0/2-2*x)
    bounds = [( max(a0/2+b0/2-a1, 0, b1/2-a1/2+a0/4+b0/4-c0/2), min(1/2.-c0/2-a1, b0/2, b1/2-a1/2+a0/4+b0/4) )]
    if bounds[0][0] > bounds[0][1]: return p_good(b0, a0, b1, a1) - 1
    return - opt.fminbound(proba, bounds[0][0], bounds[0][1], xtol=1e-15, full_output=1)[1] - 2*f(a1, b1, c1)


def p_good_2_aux(b0, a0, c0, b1, a1, c1):
    return -( 2*xlx(a1-c1) + xlx(1-2*c1-2*b1) + 2*xlx(c1) + 2*xlx(b0/2-c1) ) - 2*f(a1, b1, c1)

def H1(value):
    if value == 1.0:
        return 0.5

    # approximate inverse binary entropy function
    steps = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.0000000001, 0.0000000000001, 0.000000000000001]
    r = 0.000000000000000000000000000000001

    for step in steps:
        i = r
        while (i + step < 1.0) and (H(i) < value):
            i += step

        r = i - step

    return r

def Hi(value):
    return H1(value)

var('y')
#get H^(-1)(x)<1/2
def Hi2(v):
    if v==1:
        return 0.5
    return find_root(v-(-y*log(y,2)-(1-y)*log(1-y,2)),0.00001,0.499999)

### Adapted estimator functions

In [2]:
def _list_merge_async_complexity(L1,L2, l, hmap):
    """
    Complexity estimate of merging two lists exact

    INPUT:

    - ``L`` -- size of lists to be merged
    - ``l`` -- amount of bits used for matching
    - ``hmap`` -- indicates if hashmap is being used (Default 0: no hashmap)
    """

    if L1 == 1 and L2==1:
        return 1
    if L1==1:
        return L2
    if L2==1:
        return L1
    if not hmap:
        return 0 #to be implemented
    else:
        return L1+L2 + L1*L2 // 2 ** l
    
def _list_merge_complexity(L, l, hmap):
    """
    Complexity estimate of merging two lists exact
    INPUT:
    - ``L`` -- size of lists to be merged
    - ``l`` -- amount of bits used for matching
    - ``hmap`` -- indicates if hashmap is being used (Default 0: no hashmap)
    """

    if L == 1:
        return 1
    if not hmap:
        return max(1, 2 * int(log2(L)) * L + L ** 2 // 2 ** l)
    else:
        return 2 * L + L ** 2 // 2 ** l

    
def _gaussian_elimination_complexity(n, k, r):
    """
    Complexity estimate of Gaussian elimination routine
    INPUT:
    - ``n`` -- Row additons are perfomed on ``n`` coordinates
    - ``k`` -- Matrix consists of ``n-k`` rows
    - ``r`` -- Blocksize of method of the four russian for inversion, default is zero
    [Bar07]_ Bard, G.V.: Algorithms for solving linear and polynomial systems of equations over finite fields
    with applications to cryptanalysis. Ph.D. thesis (2007)
    [BLP08] Bernstein, D.J., Lange, T., Peters, C.: Attacking and defending the mceliece cryptosystem.
    In: International Workshop on Post-Quantum Cryptography. pp. 31–46. Springer (2008)
    """

    if r != 0:
        return (r ** 2 + 2 ** r + (n - k - r)) * int(((n + r - 1) / r))

    return (n - k) ** 2


def _optimize_m4ri(n, k, mem=inf):
    """
    Find optimal blocksize for Gaussian elimination via M4RI
    INPUT:
    - ``n`` -- Row additons are perfomed on ``n`` coordinates
    - ``k`` -- Matrix consists of ``n-k`` rows
    """

    (r, v) = (0, inf)
    for i in range(n - k):
        tmp = log2(_gaussian_elimination_complexity(n, k, i))
        if v > tmp and r < mem:
            r = i
            v = tmp
    return r


def _mem_matrix(n, k, r):
    """
    Memory usage of parity check matrix in vector space elements
    INPUT:
    - ``n`` -- length of the code
    - ``k`` -- dimension of the code
    - ``r`` -- block size of M4RI procedure
    """
    return n - k + 2 ** r

### BCJ

In [3]:
"""
Optimization target: new algorithm for subset-sum, using {0,-1,1} representations
with a 4-level merging tree.
"""
#from macros import round_to_str, check_constraints, round_upwards_to_str, wrap
import collections
from math import*
import scipy.optimize as opt
import matplotlib.pyplot as plt
import numpy as np

set_bcj = collections.namedtuple('bcj', 'l1 l2 l3 L1 L2 L3 L4 alpha1 alpha2 alpha3 r1 r2 r3')
def bcj(f) : return wrap(f, set_bcj)

def g(a, b):
    return -xlx(a) - xlx(b) - xlx(1-a-b)

p3 = lambda x: 1/4  + x.alpha3
p2 = lambda x: 1/8  + x.alpha3/2 + x.alpha2
p1 = lambda x: 1/16 + x.alpha3/4 + x.alpha2/2+x.alpha1
p0 = lambda x: p1(x)/2

m3 = lambda x: x.alpha3
m2 = lambda x: x.alpha3/2 + x.alpha2
m1 = lambda x: x.alpha3/4 + x.alpha2/2 + x.alpha1
m0 = lambda x: m1(x)/2

D1 = lambda x: g(p1(x), m1(x))
D2 = lambda x: g(p2(x), m2(x))
D3 = lambda x: g(p3(x), m3(x))
q2 = lambda x: D2(x) + x.r1 - 2*D1(x)
q3 = lambda x: D3(x) + x.r2 - 2*D2(x)
q4 = lambda x:  1    + x.r3 - 2*D3(x)

t1= lambda x: max(7*x.l1 - r(x),0)
t2= lambda x: max(7*x.l1 + 3*x.l2 - r(x),0) - t1(x)
t3= lambda x: max(7*x.l1 + 3*x.l2 + x.l3 - r(x), 0) - t1(x) - t2(x)
    
l = lambda x: x.l1+x.l2+x.l3
r = lambda x: x.r3 + 2*x.r2 + 4*x.r1

def bcj_reps(p,d,m):
    return p+d+binomH(1-p-d,m)+binomH(1-p-d-m,m)


bjc_constraints = [
# representations
{ 'type' : 'eq', 'fun' : bcj(lambda x : x.r1 - bcj_reps(p2(x),m2(x),x.alpha1) )},
{ 'type' : 'eq', 'fun' : bcj(lambda x : x.r2 - bcj_reps(p3(x),m3(x),x.alpha2) )},
{ 'type' : 'eq', 'fun' : bcj(lambda x : x.r3 - bcj_reps(1/2,0,x.alpha3) )},
    
#list sizes
#{ 'type' : 'eq'  , 'fun' : bcj(lambda x : x.L4 - (2*x.L3- (1-l(x)) +q4(x)))},
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.L3 - (2*x.L2 - x.l3 + q3(x)))},
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.L2 - (2*x.L1 -x.l2 +q2(x)))},
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.L1 - (D1(x) - x.l1))},

# correctness constraints
{ 'type' : 'ineq', 'fun' : bcj(lambda x : 1-l(x))},
# { 'type' : 'ineq', 'fun' : bcj(lambda x : x.r3-x.r2)},
# { 'type' : 'ineq', 'fun' : bcj(lambda x : x.r2-x.r1)},
    
# memory 
{ 'type' : 'ineq', 'fun' : bcj(lambda x : membound-bcj_memory(x))},
    
# saturation constraints
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.l1 - x.r1)},
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.l2 - (2*x.r1 + x.r2 - 3*x.l1))},
{ 'type' : 'ineq', 'fun' : bcj(lambda x : x.l3 - (r(x)-7*x.l1-3*x.l2))},
]

#Time with 7-Diss and repition of subtrees
def bcj_classical_time(x):
    x = set_bcj(*x)
    m=max(x.L3, x.L2, x.L1)
    space=D1(x)
    memfac=m/space
    memfac=max(1/7,min(memfac,1/4))
    timefac=time7diss(memfac)
    return max(max(space*timefac      , x.L1           ) + t1(x)
              ,max(x.L1               , x.L2 - q2(x)   ) + t2(x) + t1(x)
              ,max(x.L2, x.L3 - q3(x) , 2*x.L3-(1-l(x))) + t3(x)+ t2(x) + t1(x))

#Time with 7-Diss but without repition of subtrees
# def bjc_classical_time(x):
#     x = set_bcj(*x)
#     m=max(x.l3, x.l2, x.l1)
#     space=f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) 
#     memfac=m/space
#     memfac=max(1/7,min(memfac,1/4))
#     timefac=time7diss(memfac)
#     #return max(x.l4, x.l3, x.l2 - x.p2, x.l1 - x.p1, -x.p0)
#     it3=x.c1-x.c2 #l3
#     it2=x.c2-x.c3 #l2
#     return max(max(space*timefac     , x.l3)                  - min(2*x.l1 - (1-x.c1) + x.p0     , 0)
#               ,max(x.l3              , x.l2 - x.p2)           - min(2*x.l1 - (1-x.c1) + x.p0     , 0)
#               ,max(x.l2, x.l1 - x.p1 , 2*x.l1 - (1-x.c1))     - min(2*x.l1 - (1-x.c1) + x.p0     , 0))


#Memory with 7-Diss
def bcj_memory(x):
    x = set_bcj(*x)
    space = D1(x)
    m = max(x.L1, x.L2, x.L3)
    memfac = m/space
    memfac = max(1/7,min(memfac,1/4))
    return max(space*memfac, m)



#Time and Memory without 7-Diss and without repition of subtrees
#def bcj_classical_time(x):
#    x = set_bcj(*x)
#    return max(max(x.L4              , x.l3)                  - min(2*x.l1 - (1-x.c1) + x.p0      , 0)
#              ,max(x.l3              , x.l2 - x.p2)           - min(2*x.l1 - (1-x.c1) + x.p0      , 0)
#              ,max(x.l2, x.l1 - x.p1 , 2*x.l1 - (1-x.c1))     - min(2*x.l1 - (1-x.c1) + x.p0      , 0))

#Memory MITM
#def bcj_memory(x):
#    x = set_bcj(*x)
#    return max(x.L4, x.L3, x.L2, x.L1)


def bcj_optimize(verb=True):
    time = bcj_classical_time
    objective = time
    mycons = bjc_constraints
    
    start = [random.uniform(0,0.15) for _ in range(7)]+[random.uniform(0,0.05) for _ in range(3)]+[random.uniform(0,1) for _ in range(3)]
    bounds = [(0,0.8)]*7+[(0,0.05)]*3+[(0,1)]*3
    
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-10,
            constraints=mycons, options={'maxiter':1500}) # tune this paramter.

    astuple = set_bcj(*result.x)

    if verb:
        # print("Probas")
        # print(q2(astuple),q3(astuple),q4(astuple))
        print(t1(astuple),t2(astuple),t3(astuple))
        print("memory ", bcj_memory(result.x))
        print("Validity: ", result.success)
        print("Time: ", round_upwards_to_str(time(astuple)))
        for t in astuple._asdict():
            print(t, round_upwards_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(mycons, result.x))
        
    t = check_constraints(mycons, result.x)
    
    if all(-10**(-7)<=i[1]<=10**(-7) for i in t if i[0]=="eq") and all(-10**(-7)<=i[1] for i in t if i[0]=="ineq"):
        return time(astuple)
    else:
        return -1


membound=1
a = bcj_optimize()

0 0 0.010484402372606727
memory  0.2892096752441659
Validity:  True
Time:  2893/10000
l1 681/2500
l2 2893/10000
l3 671/2500
L1 2893/10000
L2 171/625
L3 27/125
L4 513/10000
alpha1 9/2000
alpha2 153/10000
alpha3 369/10000
r1 681/2500
r2 2669/5000
r3 8753/10000
Checking that the constraints are satisfied:
[('eq', 6.628031457012185e-14), ('eq', 8.629763570411342e-13), ('eq', 1.5184520307798266e-11), ('ineq', 1.0988709941983643e-12), ('ineq', 1.1818879208647104e-12), ('ineq', 1.3104517471163035e-12), ('ineq', 0.17004240559879458), ('ineq', 0.710790324755834), ('ineq', 2.6645352591003757e-15), ('ineq', 0.027865663441889676), ('ineq', 0.010484402372606394)]


In [5]:
# calc runtime for every memory limitation
L=[]
membound = 0.29 #0.289
while membound > 0.05:
    mini=2
    c=0
    while c<5:
        try:
            t = float(bcj_optimize(False))
        except(ValueError):
            print("error")
            continue

        if t!=-1:
            if mini>t:
                mini=t
            c+=1

    L.append([membound,mini])
    membound -= 0.02

In [6]:
L

[[0.290000000000000, 0.28916118664900803],
 [0.270000000000000, 0.29448540642051513],
 [0.250000000000000, 0.31330809892124745],
 [0.230000000000000, 0.3339102477753666],
 [0.210000000000000, 0.35520266267161626],
 [0.190000000000000, 0.3773210727986046],
 [0.170000000000000, 0.41167258765984244],
 [0.150000000000000, 0.46119318511384627],
 [0.130000000000000, 0.5115353679521613],
 [0.110000000000000, 0.5629397456378887],
 [0.0900000000000000, 0.615833619835171],
 [0.0700000000000000, 0.6710624430475025],
 [0.0500000000000000, 0.736616933313797]]

### BBSS

In [24]:
set_bbss = collections.namedtuple('classical', 'p0 p1 p2 l1 l2 l3 l4 c1 c2 c3 alpha1 alpha2 alpha3 gamma1 gamma2 gamma3')
def bbss(f) : return wrap(f, set_bbss)

#=================================
# Among these variables:
# - pi is the filtering probability at level i
# - li is the list size at level i (classically, all lists at a given level
# have same size)
# (l0 = 0, since we want to find the solution)
# - ci is the total number of bits of the modular constraint at level i
# - alphai is the total number of "-1" at level i
# - gammai is the total number of "2" at level i

constraints_bbss = [
    # filtering terms
    { 'type' : 'eq', 'fun' : bbss(lambda x : p_good_2_aux(1/2., 0, 0., 1/4.+x.alpha1-2*x.gamma1, x.alpha1, x.gamma1) - x.p0 )},
    { 'type' : 'eq', 'fun' : bbss(lambda x : p_good_2(1/4.+x.alpha1-2*x.gamma1, x.alpha1, x.gamma1, 1/8.+x.alpha2-2*x.gamma2, x.alpha2, x.gamma2) - x.p1)},
    { 'type' : 'eq', 'fun' : bbss(lambda x : p_good_2(1/8.+x.alpha2-2*x.gamma2, x.alpha2, x.gamma2, 1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) - x.p2)},
    # sizes of the lists
    #{ 'type' : 'eq', 'fun' : bbss(lambda x : 2*x.l1 - (1-x.c1) + x.p0 )},
    { 'type' : 'eq', 'fun' : bbss(lambda x : 2*x.l2 - (x.c1 - x.c2) + x.p1 - x.l1 )},
    { 'type' : 'eq', 'fun' : bbss(lambda x : 2*x.l3 - (x.c2 - x.c3) + x.p2 - x.l2 )},
    { 'type' : 'eq', 'fun' : bbss(lambda x : 2*x.l4 - x.c3 - x.l3 )},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : f(1/4.+x.alpha1-2*x.gamma1, x.alpha1, x.gamma1) - x.c1 - x.l1)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : f(1/8.+x.alpha2-2*x.gamma2, x.alpha2, x.gamma2) - x.c2 - x.l2)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) - x.c3 - x.l3)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.l4-f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3)*0.5 )},
    # coherence of the -1
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.alpha2 - x.alpha1/2)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.alpha3 - x.alpha2/2)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.alpha1 - 2*x.gamma1)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.alpha2 - 2*x.gamma2)},
    { 'type' : 'ineq', 'fun' : bbss(lambda x : x.alpha3 - 2*x.gamma3)},
    #memory 
    { 'type' : 'ineq', 'fun' : bbss(lambda x : membound-bbss_memory(x))},
    
    # if no 2
    #{ 'type' : 'ineq', 'fun' : bbss(lambda x : - x.gamma1)},
    #{ 'type' : 'ineq', 'fun' : bbss(lambda x : - x.gamma2)},
    #{ 'type' : 'ineq', 'fun' : bbss(lambda x : - x.gamma3)},
]

#Time with 7-Diss and repition of subtrees
def bbss_time(x):
    x = set_bbss(*x)
    m=max(x.l3, x.l2, x.l1)
    space=f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) 
    memfac=m/space
    memfac=max(1/7,min(memfac,1/4))
    timefac=time7diss(memfac)
    it3=x.c1-x.c2 #l3
    it2=x.c2-x.c3 #l2
    return max(max(space*timefac     , x.l3)                  - min(2*x.l1 - (1-x.c1) + x.p0 +it3 + 3*it2     , 0)
              ,max(x.l3              , x.l2 - x.p2)           - min(2*x.l1 - (1-x.c1) + x.p0 +it3             , 0)
              ,max(x.l2, x.l1 - x.p1 , 2*x.l1 - (1-x.c1))     - min(2*x.l1 - (1-x.c1) + x.p0                  , 0))

#Time with 7-Diss but without repition of subtrees
#def bbss_time(x):
#    x = set_bbss(*x)
#    m=max(x.l3, x.l2, x.l1)
#    space=f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) 
#    memfac=m/space
#    memfac=max(1/7,min(memfac,1/4))
#    timefac=time7diss(memfac)
#    #return max(x.l4, x.l3, x.l2 - x.p2, x.l1 - x.p1, -x.p0)
#    it3=x.c1-x.c2 #l3
#    it2=x.c2-x.c3 #l2
#    return max(max(space*timefac     , x.l3)                  - min(2*x.l1 - (1-x.c1) + x.p0     , 0)
#              ,max(x.l3              , x.l2 - x.p2)           - min(2*x.l1 - (1-x.c1) + x.p0     , 0)
#              ,max(x.l2, x.l1 - x.p1 , 2*x.l1 - (1-x.c1))     - min(2*x.l1 - (1-x.c1) + x.p0     , 0))


#Memory with 7-Diss
def bbss_memory(x):
    x = set_bbss(*x)
    space = f(1/16.+x.alpha3-2*x.gamma3, x.alpha3, x.gamma3) 
    m = max(x.l3, x.l2, x.l1)
    memfac = m/space
    memfac = max(1/7,min(memfac,1/4))
    return max(space * memfac, x.l3, x.l2, x.l1)



#Time and Memory without 7-Diss and without repition of subtrees
#def bbss_time(x):
#    x = set_bbss(*x)
#    it3=x.c1-x.c2 #l3 
#    it2=x.c2-x.c3 #l2
#    return max(max(x.l4              , x.l3)                  - min(2*x.l1 - (1-x.c1) + x.p0      , 0) #+it3 + 3*it2
#              ,max(x.l3              , x.l2 - x.p2)           - min(2*x.l1 - (1-x.c1) + x.p0      , 0)# +it3
#              ,max(x.l2, x.l1 - x.p1 , 2*x.l1 - (1-x.c1))     - min(2*x.l1 - (1-x.c1) + x.p0      , 0))

#Memory
#def bbss_memory(x):
#    x = set_classical(*x)
#    return max(x.l4, x.l3, x.l2, x.l1)

def optimize_bbss(verb=True):
    time = bbss_time
    objective = time
    mycons = constraints_bbss
    
    start = [(-0.2)]*3 + [random.uniform(0.18,0.22) for _ in range(7)] + [0.03]*3 + [(0.000)]*3
    bounds = [(-0.6,0)]*3 + [(0,1)]*7 + [(0, 0.05)]*3 + [(0, 0.00)]*3
    
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-10,
            constraints=mycons, options={'maxiter':1000})
    
    astuple = set_bbss(*result.x)
    if verb:
        print("memory ", bbss_memory(result.x))
        print("Validity: ", result.success)
        print("Time: ", round_upwards_to_str(time(astuple)))
        for t in astuple._asdict():
            print(t, round_upwards_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(mycons, result.x))
    t=check_constraints(mycons, result.x)
    if all(-10**(-7)<=i[1]<=10**(-7) for i in t if i[0]=="eq") and all(-10**(-7)<=i[1] for i in t if i[0]=="ineq"):
        return time(astuple)
    else:
        return -1
    #return result.success, objective(astuple), result
    #return time(astuple)

membound=1
optimize_bbss()

memory  0.3081813514562547
Validity:  True
Time:  1607/5000
p0 -3213/10000
p1 -447/10000
p2 -29/2500
l1 1339/5000
l2 1541/5000
l3 1539/5000
l4 2451/10000
c1 391/500
c2 2391/5000
c3 114/625
alpha1 1/20
alpha2 269/10000
alpha3 17/1250
gamma1 0000
gamma2 0000
gamma3 0000
Checking that the constraints are satisfied:
[('eq', -2.7755575615628914e-16), ('eq', -3.977998486170975e-13), ('eq', -1.834052007487763e-12), ('eq', -5.551115123125783e-17), ('eq', -5.551115123125783e-17), ('eq', -5.551115123125783e-17), ('ineq', 0.09145877512133282), ('ineq', 0.0003636070140848324), ('ineq', 1.1102230246251565e-16), ('ineq', -8.326672684688674e-17), ('ineq', 0.0018949177017651316), ('ineq', 0.00011146229144144877), ('ineq', 0.049999930308749795), ('ineq', 0.02689488285614003), ('ineq', 0.013558903719511463), ('ineq', 0.6918186485437453)]


0.3213427552078553

### optimize for all memory limits

In [25]:
# calc runtime for every memory limitation
L=[]
membound = 0.29 #0.289
while membound >0.05:
    mini=2
    c=0
    while c<5:
        try:
            t = float(optimize_bbss(False))
        except(ValueError):
            print("error")
            continue

        if t!=-1:
            if mini>t:
                mini=t
            c+=1

    L.append([membound,mini])
    membound-=0.02

In [26]:
L

[[0.290000000000000, 0.2891460775716504],
 [0.270000000000000, 0.2944853869737913],
 [0.250000000000000, 0.31430849859090626],
 [0.230000000000000, 0.3339101394546195],
 [0.210000000000000, 0.35520265339329293],
 [0.190000000000000, 0.3773210702665528],
 [0.170000000000000, 0.4116725924468787],
 [0.150000000000000, 0.4611938919115602],
 [0.130000000000000, 0.5116345491615812],
 [0.110000000000000, 0.5650324350976486],
 [0.0900000000000000, 0.6158468588571047],
 [0.0700000000000000, 0.6712109978603655],
 [0.0500000000000000, 0.7366171719995305]]

### Code based optimisations

In [7]:
# number of repitions each optimisation is run to find the optimal value.
retries = 10

### BJMM

In [4]:
### BJMM
k = 0.448 #0.426 FD
w = Hi(1 - k)/2

In [7]:
set_bjmm = collections.namedtuple('BJMM', 'l p p1 p2 L0 L1 L2 r1 r2')
def bjmm(f) : return wrap(f, set_bjmm)
def bjmm_reps(p, p2, l): 
    if p == 0. or p2 == 0. or l == 0.:
        return 0
    if l < p2 or p < p2/2. or l - p2 < p - p2/2.:
        return 0.
    
    return binomH(p2, p2/2.) + binomH(l-p2, p-p2/2.) 

def classical_time_bjmm(x):
    x = set_bjmm(*x)    
    perms = binomHH(1., w) - binomHH(k + x.l, x.p) - binomHH(1. - k - x.l, w - x.p)
    T1 = max(2.*x.L0 - x.r1, x.L0);
    T2 = max(2.*x.L1 - (x.r2 - x.r1), x.L1)
    T3 = max(2.*x.L2 - (x.l - x.r2), x.L2)
    return perms + max(T1, T2, T3)


def classical_mem_bjmm(x):
    x = set_bjmm(*x)  
    return max(x.L0, x.L1, x.L2)

constraints_bjmm = [
    # weights    
    { 'type' : 'ineq',   'fun' : bjmm(lambda x : (2. * x.p1) - x.p2) }, 
    { 'type' : 'ineq',   'fun' : bjmm(lambda x : (2. * x.p2) - x.p) }, 

    # representations and constrains
    { 'type' : 'ineq',   'fun' : bjmm(lambda x : x.r2 - x.r1) }, 
    { 'type' : 'ineq',   'fun' : bjmm(lambda x : x.l - x.r2) },
    
    # reps
    { 'type' : 'eq',   'fun' : bjmm(lambda x : x.r1 - bjmm_reps(x.p1, x.p2, k + x.l)) }, 
    { 'type' : 'eq',   'fun' : bjmm(lambda x : x.r2 - bjmm_reps(x.p2, x.p,  k + x.l)) }, 

    # list
    { 'type' : 'eq',   'fun' : bjmm(lambda x : x.L0 -  binomHH((k+x.l)/2, x.p1/2.)) },
    { 'type' : 'eq',   'fun' : bjmm(lambda x : x.L1 - (binomHH((k+x.l), x.p1) - x.r1)) },
    { 'type' : 'eq',   'fun' : bjmm(lambda x : x.L2 - (binomHH((k+x.l), x.p2) - x.r2)) },
    
    # memory 
    { 'type' : 'ineq', 'fun' : bjmm(lambda x : bjmm_membound-classical_mem_bjmm(x))},
]



def optimize_bjmm(verb=True):
    time = classical_time_bjmm
    objective = time
    mycons = constraints_bjmm
    
    start = [(rng.uniform(0.05,0.09))]+[(rng.uniform(0.01,0.02))] + [(rng.uniform(0.001,0.015))]*2 + [(0.031)]*3 + [(rng.uniform(0.001,0.2))]*2
    bounds = [(0.05, 0.08)]*1 + [(0.001, 0.03)]*1 + [(0.0002, 0.02)]*2 + [(0.001, 0.04)]*3 + [(0.002, 0.05)]*2
    
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-7, 
            constraints=mycons, options={'maxiter':10000})
    
    astuple = set_bjmm(*result.x)
    
    if verb:
        print("Validity: ", result.success)
        print("Time: ", time(astuple))
        
        for t in astuple._asdict():
            print(t, round_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(mycons, result.x))

    return result.success, objective(astuple), result

def optimize_mem_bjmm():  
    """
    finds the optimal runtime under a certain memory restriction
    """
    mmt_data = []
    for mem in float_range(0., 0.2, 0.1):
        global bjmm_membound
        bjmm_membound = mem
        time = min([optimize_bjmm(False) for _ in range(retries)])
        mmt_data.append([mem, time])
    return mmt_data

def optimize_k_bjmm():
    """
    optimize under restrictions on the code rate
    """
    mmt_data = []
    for k_ in float_range(0., 0.5, 0.1):
        global k
        global w
        k = k_
        w = Hi(1 - k_)/2 # half distance
        time = min([optimize_bjmm(False) for _ in range(retries)])
        mmt_data.append([k, w, time])
    return mmt_data

bjmm_membound = 1
optimize_bjmm()

Validity:  False
Time:  0.06564865883679777
l 0.0500
p 0.0012
p1 0.0002
p2 0.0004
L0 0.0025
L1 0.0038
L2 0.0057
r1 0.0020
r2 0.0020
Checking that the constraints are satisfied:
[('ineq', 2.1275353440791678e-14), ('ineq', -0.00037149976488334217), ('ineq', -3.665642442363115e-12), ('ineq', 0.04800000000000008), ('eq', 0.0015999893485353861), ('eq', 0.0020000000000000894), ('eq', 0.001227245216927316), ('eq', 0.0032144904338546674), ('eq', 0.0029904262842324458), ('ineq', 0.9943198459713214)]


(False,
 0.06564865883679777,
      fun: 0.06564865883679777
      jac: array([ 0.18954798, -5.92326127,  0.        ,  0.        ,  0.        ,
         2.        ,  0.        ,  1.        , -1.        ])
  message: 'Positive directional derivative for linesearch'
     nfev: 125
      nit: 12
     njev: 8
   status: 8
  success: False
        x: array([0.05      , 0.00117152, 0.00020001, 0.00040001, 0.00249971,
        0.00375942, 0.00568015, 0.002     , 0.002     ]))

In [ ]:
optimize_mem_bjmm()

In [33]:
optimize_k_bjmm()

[[0.0,
  0.250000000000000,
  (False,
   0.013457753453042877,
        fun: 0.013457753453042877
        jac: array([ 6.48200512e-07, -1.07288361e-06,  0.00000000e+00,  0.00000000e+00,
           1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00])
    message: 'Iteration limit reached'
       nfev: 196995
        nit: 10000
       njev: 10000
     status: 9
    success: False
          x: array([0.05      , 0.01249999, 0.00615683, 0.01231366, 0.01345775,
          0.001     , 0.00382755, 0.02591551, 0.03643832]))],
 [0.1,
  0.1580096731618021,
  (False,
   0.02198890216854617,
        fun: 0.02198890216854617
        jac: array([ 1.09240599e-01, -8.09863061e-01,  0.00000000e+00,  0.00000000e+00,
           2.00000000e+00,  9.99999992e-01,  0.00000000e+00, -8.14907253e-09,
           0.00000000e+00])
    message: 'Iteration limit reached'
       nfev: 197607
        nit: 10000
       njev: 10000
     status: 9
    success: False
          x: ar

### New BJMM

In [10]:
k = 0.448 #0.426 FD
w = Hi(1 - k)/2

In [14]:
set_new_bjmm = collections.namedtuple('NEWBJMM', 'l p p1 p2 L1 L2 L3 r1 r2 l1 l2')
def new_bjmm(f) : return wrap(f, set_new_bjmm)
def new_bjmm_reps(p, p2, l): 
    if p == 0. or p2 == 0. or l == 0.:
        return 0
    if l < p2 or p < p2/2. or l - p2 < p - p2/2.:
        return 0.
    
    return binomHH(p2, p2/2.) + binomHH(l-p2, p-p2/2.) 

def classical_time_new_bjmm(x):
    x = set_new_bjmm(*x)    
    perms = binomHH(1., w) - binomHH(k + x.l, x.p) - binomHH(1. - k - x.l, w - x.p)
    T1 = max(2*x.L1 - x.l1, x.L1)
    T2 = max(2*x.L2 - x.r2, x.L2)
    T3 = max(2*x.L3 - (x.l - x.l2 - x.l1), x.L3)
    return max(
            max(3*x.l1 - 2*x.r1 - x.r2, 0) + T1,
            max(3*x.l1 + x.l2 - 2*x.r1 - x.r2, 0) + max(T2, T3)
    ) + perms
    #return perms + max(x.L1, 
    #                   2*x.L1-x.r1, x.L2, 
    #                   2*x.L2-(x.r2-x.r1), 
    #                   x.L3, 
    #                   2*x.L3 - (x.l - x.r2)) 
    #                   #x.L4) 


def classical_mem_new_bjmm(x):
    x = set_new_bjmm(*x)  
    return max(x.L1/2, x.L2, x.L3)#, x.L4)

constraints_new_bjmm = [
    # weights    
    { 'type' : 'ineq',   'fun' : new_bjmm(lambda x : 2. * x.p1 - x.p2) }, 
    { 'type' : 'ineq',   'fun' : new_bjmm(lambda x : 2. * x.p2 - x.p) }, 

    # reps
    { 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.r1 - new_bjmm_reps(x.p1, x.p2, k + x.l)) }, 
    { 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.r2 - new_bjmm_reps(x.p2, x.p , k + x.l)) }, 

    # binomial coeeficient correctness
    { 'type' : 'ineq',   'fun' : new_bjmm(lambda x : x.l - (x.l1 + x.l2)) }, 
    { 'type' : 'ineq',   'fun' : new_bjmm(lambda x : x.l1 - x.r1 ) }, 
    { 'type' : 'ineq',   'fun' : new_bjmm(lambda x : x.l2 - x.r2 ) }, 
    #{ 'type' : 'ineq',   'fun' : new_bjmm(lambda x : 2*x.r1 + x.r2 - x.l2 - 3*x.l1) }, 

    # list
    { 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.L1 -  binomHH((k+x.l)/2, x.p1/2.)) },
    { 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.L2 - (binomHH((k+x.l), x.p1) - x.l1)) },
    { 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.L3 - (binomHH((k+x.l), x.p2) + x.r1 - 3*x.l1 - x.l2)) },
    #{ 'type' : 'eq',   'fun' : new_bjmm(lambda x : x.L4 - (binomHH((k+x.l), x.p) - x.l)  ) },

    # correctness
    #{ 'type' : 'ineq',   'fun' : new_bjmm(lambda x : 1. - k - x.l) }, 
    #{ 'type' : 'ineq',   'fun' : new_bjmm(lambda x : (1. - k - x.l) - (w - x.p)) }, 
    #{ 'type' : 'ineq',   'fun' : new_bjmm(lambda x : w - x.p) }, 

    # memory 
    { 'type' : 'ineq', 'fun' : new_bjmm(lambda x : new_bjmm_membound-classical_mem_new_bjmm(x))},
]


def optimize_new_bjmm(verb=True):
    time = classical_time_new_bjmm
    objective = time
    mycons = constraints_new_bjmm
    
    start = [(rng.uniform(0.06,0.08))]+[(rng.uniform(0.01,0.02))] + [(rng.uniform(0.005,0.2))]*9
    bounds = [(0.06, 0.08)]*1 +  [(0.01, 0.02)]*1 +[(0.0002, 0.02)]*2 + [(0.001, 0.04)]*7
    
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-7, 
            constraints=mycons, options={'maxiter':10000})
    
    astuple = set_new_bjmm(*result.x)
    if verb:
        print("Validity: ", result.success)
        print("Time: ", time(astuple))
        
        for t in astuple._asdict():
            print(t, round_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(mycons, result.x))

    return result.success, objective(astuple), result

def optimize_mem_new_bjmm():  
    """
    finds the optimal runtime under a certain memory restriction
    """
    bjmm_data = []
    for mem in float_range(0., 0.2,  0.02):
        global new_bjmm_membound
        new_bjmm_membound = mem
        time = min([optimize_new_bjmm(False) for _ in range(retries)])
        bjmm_data.append([mem, time])
    return bjmm_data

def optimize_k_new_bjmm():
    """
    optimize under restrictions on the code rate
    """
    bjmm_data = []
    for k_ in float_range(0., 0.5, 0.1):
        global k
        global w
        k = k_
        w = Hi(1 - k_)/2 # half distance
        time = min([optimize_new_bjmm(False) for _ in range(retries)])
        bjmm_data.append([k, w, time])
    return bjmm_data

new_bjmm_membound = 1
optimize_new_bjmm()

/usr/lib/python3.10/site-packages/scipy/optimize/_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Validity:  True
Time:  0.05968853905457788
l 0.0600
p 0.0165
p1 0.0048
p2 0.0087
L1 0.0196
L2 0.0213
L3 0.0025
r1 0.0139
r2 0.0213
l1 0.0178
l2 0.0213
Checking that the constraints are satisfied:
[('ineq', 0.0009144859158076984), ('ineq', 0.0008159479531300549), ('eq', 7.554219402794438e-11), ('eq', 1.4476961296416846e-13), ('ineq', 0.02089675875774278), ('ineq', 0.0038890590206432686), ('ineq', -3.8007791358651843e-14), ('eq', 3.5029167066991107e-12), ('eq', 7.0058299439512695e-12), ('eq', -7.568485768660871e-13), ('ineq', 0.9787241508218252)]


(True,
 0.05968853905457788,
      fun: 0.05968853905457788
      jac: array([ 0.09880508, -1.66940016,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        , -2.        , -1.        ,  3.        ,
         1.        ])
  message: 'Optimization terminated successfully'
     nfev: 7946
      nit: 563
     njev: 562
   status: 0
  success: True
        x: array([0.06      , 0.01651216, 0.00478927, 0.00866405, 0.0195516 ,
        0.02127585, 0.00246229, 0.01393829, 0.02127589, 0.01782735,
        0.02127589]))

In [37]:
# optimizer over all memory limitations
optimize_mem_new_bjmm()

[[0.0,
  (False, 0.05098448144377801,      fun: 0.05098448144377801
        jac: array([ 0.13908689, -2.6185398 ,  0.        ,  0.        ,  1.        ,
           0.        ,  0.        , -2.        , -1.        ,  3.        ,
           0.        ])
    message: 'Positive directional derivative for linesearch'
       nfev: 160
        nit: 15
       njev: 11
     status: 8
    success: False
          x: array([0.06      , 0.01000015, 0.00229611, 0.00459221, 0.00911304,
          0.00194024, 0.00100037, 0.01628584, 0.001     , 0.01628584,
          0.001     ]))],
 [0.02,
  (False, 0.058264217741284915,      fun: 0.058264217741284915
        jac: array([ 0.10049487, -1.70345418,  0.        ,  0.        ,  0.        ,
           1.        ,  0.        , -2.        , -1.        ,  3.        ,
           1.        ])
    message: 'Iteration limit reached'
       nfev: 120319
        nit: 10000
       njev: 10000
     status: 9
    success: False
          x: array([0.06      , 0.0162387

In [38]:
# optimize over all code rates
optimize_k_new_bjmm()

[[0.0,
  0.250000000000000,
  (False,
   0.016845342715837132,
        fun: 0.016845342715837132
        jac: array([ 0.16227146, -0.77760614,  0.        ,  0.        ,  0.        ,
           1.9062628 ,  0.        , -2.        , -1.        ,  3.        ,
           1.        ])
    message: 'Iteration limit reached'
       nfev: 219822
        nit: 10000
       njev: 10000
     status: 9
    success: False
          x: array([0.06      , 0.01      , 0.0025    , 0.005     , 0.00749647,
          0.00996974, 0.00481286, 0.00502319, 0.00996974, 0.00502321,
          0.00996974]))],
 [0.1,
  0.1580096731618021,
  (False,
   0.029912785979381903,
        fun: 0.029912785979381903
        jac: array([ 0.18652706, -1.6818148 ,  0.        ,  0.        ,  0.        ,
           1.25667961,  0.        , -2.        , -1.        ,  3.        ,
           1.        ])
    message: 'Iteration limit reached'
       nfev: 219672
        nit: 10000
       njev: 10000
     status: 9
    success: False

### MMT

In [ ]:
set_mmt = collections.namedtuple('MMT', 'l p L1 L2 L3 r1')
def mmt(f) : return wrap(f, set_mmt)

# Memory with 7-Diss
def dissection_mmt_memory(x):
    x = set_mmt(*x)
    space = binomHH((k+x.l)/2, x.p/4)
    m = max(x.L1, x.L2, x.L3)
    memfac = m / space
    memfac = max(1/7,min(memfac,1/4))
    return max(space * memfac, m)

def classical_time_mmt(x):
    x = set_mmt(*x)
    perms = binomHH(1., w) - binomHH(k + x.l, x.p) - binomHH(1. - k - x.l, w - x.p)
    return perms + max(x.L1, x.L2, x.L3)

def classical_memory_mmt(x):
    return max(x.L1, x.L2, x.L3)
    #return max(x.L1/2, x.L2, x.L3)
    #return dissection_mmt_memory(x)

# classical mmt
constraints_mmt = [
    # list
    { 'type' : 'eq',   'fun' : mmt(lambda x : x.L1 - binomHH((k+x.l)/2, x.p/4)) },
    { 'type' : 'eq',   'fun' : mmt(lambda x : x.L2 - (2*x.L1 - x.r1)) },
    { 'type' : 'eq',   'fun' : mmt(lambda x : x.L3 - (2*x.L2 - (x.l - x.r1))) },
    # reps
    { 'type' : 'eq',   'fun' : mmt(lambda x : x.r1 - binomHH(x.p, x.p/2.)) }, 
    # memory 
    { 'type' : 'ineq', 'fun' : mmt(lambda x : mmt_membound-classical_memory_mmt(x))},
]


def optimize_mmt(verb=True):    
    start = [(rng.uniform(0.001,0.2))]*2 + [(0.1)]*3 + [(rng.uniform(0.001,0.22))]*1
    bounds = [(0., 0.3)]*6
    
    result = opt.minimize(classical_time_mmt, start, 
            bounds= bounds, tol=1e-7, 
            constraints=constraints_mmt, options={'maxiter':10000})
    
    astuple = set_mmt(*result.x)
    
    if verb:
        print("Validity: ", result.success)
        print("Time: ", classical_time_mmt(astuple))
        
        for t in astuple._asdict():
            print(t, round_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(constraints_mmt, result.x))

        return result.success, classical_time_mmt(astuple), result
    return classical_time_mmt(astuple)#, classical_memory_mmt(astuple)

def optimize_mem_mmt():  
    """
    finds the optimal runtime under a certain memory restriction
    """
    mmt_data = []
    for mem in float_range(0., 0.2, 0.02):
        global mmt_membound
        mmt_membound = mem
        time = min([optimize_mmt(False) for _ in range(retries)])
        mmt_data.append([mem, time])
    return mmt_data

def optimize_k_mmt():
    """
    optimize under restrictions on the code rate
    """
    mmt_data = []
    for k_ in float_range(0., 0.5, 0.1):
        global k
        global w
        k = k_
        w = Hi(1 - k_)/2
        time = min([optimize_mmt(False) for _ in range(retries)])
        mmt_data.append([k, w, time])
    return mmt_data

mmt_membound = 1
optimize_mmt()

In [41]:
mmt_mem_data = optimize_mem_mmt()

/tmp/ipykernel_142067/1699032127.py:46: RuntimeWarning: invalid value encountered in double_scalars
  return n * HHH(k/n)


In [42]:
mmt_k_data = optimize_k_mmt()

/tmp/ipykernel_142067/1699032127.py:46: RuntimeWarning: invalid value encountered in double_scalars
  return n * HHH(k/n)
/tmp/ipykernel_142067/1699032127.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  return n * HHH(k/n)


### New MMT

In [47]:
set_new_mmt = collections.namedtuple('NEWMMT', 'l p L1 L2 L3 r1 l1')
def new_mmt(f) : return wrap(f, set_new_mmt)

# Memory with 7-Diss
def dissection_new_mmt_memory(x):
    x = set_new_mmt(*x)
    space = binomHH((k+x.l)/2, x.p/4)
    m = max(x.L1, x.L2, x.L3)
    memfac = m / space
    memfac = max(1/7, min(memfac,1/4))
    return max(space * memfac, m)

def classical_time_new_mmt(x):
    x = set_new_mmt(*x)
    perms = binomHH(1., w) - binomHH(k + x.l, x.p) - binomHH(1. - k - x.l, w - x.p)
    tree = max(x.L1, x.L2, x.L3) + max(0, x.l1-x.r1)
    return perms + tree

def classical_memory_new_mmt(x):
    x = set_new_mmt(*x)
    return max(x.L1, x.L2, x.L3)
    #return max(x.L1/2, x.L2, x.L3)
    #return dissection_genius_mmt_memory(x)

constraints_new_mmt = [
    # list
    { 'type' : 'eq',   'fun' : new_mmt(lambda x : x.L1 - binomHH((k+x.l)/2, x.p/4)) },
    { 'type' : 'eq',   'fun' : new_mmt(lambda x : x.L2 - (2*x.L1 - x.l1)) },
    { 'type' : 'eq',   'fun' : new_mmt(lambda x : x.L3 - (2*x.L2 - (x.l - x.l1))) },
    # reps
    { 'type' : 'eq',   'fun' : new_mmt(lambda x : x.r1 - binomHH(x.p, x.p/2.)) }, 
    # memory 
    { 'type' : 'ineq', 'fun' : new_mmt(lambda x : mmt_new_membound-classical_memory_new_mmt(x))},
    # correctness
    { 'type' : 'ineq', 'fun' : new_mmt(lambda x : x.l - x.l1) },
    { 'type' : 'ineq', 'fun' : new_mmt(lambda x : x.l1 - x.r1) }, 

]

def optimize_new_mmt(verb=True):
    start = [(rng.uniform(0.0, 0.03))]*1 + [(rng.uniform(0.0, 0.03))]*1 + [(0.01)]*3 + [(rng.uniform(0.001,0.01))]*2
    #start = [(0.022)]*1 + [(0.007)]*1 + [(0.015)]*3 + [(0.007)]*2
    bounds = [(0., 0.05)] + [(0., 0.05)] + [(0., 0.02)]*3 + [(0., 0.02)]*2
    
    result = opt.minimize(classical_time_new_mmt, start, 
            bounds= bounds, tol=1e-7, 
            constraints=constraints_new_mmt, options={'maxiter':1000})
    
    astuple = set_new_mmt(*result.x)
    
    if verb:
        print("Validity: ", result.success)
        print("Time: ", classical_time_new_mmt(astuple))
        
        for t in astuple._asdict():
            print(t, round_to_str(astuple._asdict()[t]) )
        print("Checking that the constraints are satisfied:")
        print(check_constraints(constraints_new_mmt, result.x))

        return result.success, classical_time_new_mmt(astuple), result
    return classical_time_new_mmt(astuple)

def optimize_mem_new_mmt():
    """
    finds the optimal runtime under a certain memory restriction
    """
    mmt_data = []
    retries = 3
    for mem in float_range(0., 0.2, 0.02):
        global mmt_new_membound
        mmt_new_membound = mem
        time = min([optimize_new_mmt(False) for _ in range(retries)])
        mmt_data.append([mem, time])
    return mmt_data

def optimize_k_new_mmt():
    """
    optimize under restrictions on the code rate
    """
    mmt_data = []
    retries = 3
    for k_ in float_range(0., 0.5, 0.1):
        global k
        global w
        k = k_
        w = Hi(1 - k_)/2
        
        time = min([optimize_new_mmt(False) for _ in range(retries)])

        mmt_data.append([k, w, time])
    return mmt_data

mmt_new_membound=1
optimize_new_mmt()

Validity:  True
Time:  -17669050.04000003
l 0.0500
p 0.0032
L1 0.0200
L2 0.0200
L3 0.0100
r1 0.0032
l1 0.0200
Checking that the constraints are satisfied:
[('eq', -8.535569126499887e-11), ('eq', -3.089194251446048e-08), ('eq', -3.133484961261035e-08), ('eq', 0.0), ('ineq', 0.98), ('ineq', 0.030000030891942517), ('ineq', 0.016777991280935056)]


(True,
 -17669050.04000003,
      fun: -17669050.04000003
      jac: array([-1.00e+03, -2.25e+01,  2.50e-01, -0.00e+00,  0.00e+00, -1.00e+00,
         1.00e+00])
  message: 'Optimization terminated successfully'
     nfev: 49
      nit: 7
     njev: 6
   status: 0
  success: True
        x: array([0.05      , 0.00322198, 0.02      , 0.02      , 0.00999994,
        0.00322198, 0.01999997]))

In [48]:
genius_mmt_mem_data = optimize_mem_new_mmt()

/tmp/ipykernel_142067/1699032127.py:46: RuntimeWarning: invalid value encountered in double_scalars
  return n * HHH(k/n)


In [49]:
genius_mmt_k_data = optimize_k_new_mmt()

/tmp/ipykernel_142067/1699032127.py:46: RuntimeWarning: invalid value encountered in double_scalars
  return n * HHH(k/n)


### Table generations of bit security

In [14]:
def bjmm_depth_2_qc_complexity(n, k, w, mem=inf, hmap=1, mmt=0, qc=0, base_p=-1, l_val=0, l1_val=0, memory_access=0, enable_tmto=1):
    n = int(n)
    k = int(k)
    w = int(w)
    
    solutions = max(0, log2(binom(n, w)) - (n - k))
    time = inf
    memory = 0
    r = _optimize_m4ri(n, k, mem)

    i_val = [25, 450, 25]
    i_val_inc = [10, 10, 10]
    params = [-1 for _ in range(7)]
    lists=[]

    while True:
        stop = True
        
        mod2 = (params[0] - i_val_inc[0]//2) % 2
        for p in range(max(params[0] - i_val_inc[0]//2 - mod2, 2*qc), min(w // 2, i_val[0]), 2):
            for l in range(max(params[1] - i_val_inc[1] // 2, 0), min(n - k - (w - 2 * p), min(i_val[1], n - k))):
                for p1 in range(max(params[2] - i_val_inc[2] // 2, (p + 1) // 2,qc), min(w, i_val[2])):
                    if mmt and p1 != p // 2:
                        continue
                        
                    if base_p!=-1 and p1!=base_p:
                        continue
                        
                    if l_val!=0 and l!=l_val:
                        continue
                        
                        
                    k1 = (k + l) // 2
                    L1 = binom(k1, p1)
                    if qc:
                        L1b = binom(k1, p1-1)*k
                        
                    if log2(L1) > time:
                        continue

                    if k1 - p < p1 - p / 2:
                        continue
                    
                    if not(qc):
                        reps = (binom(p, p//2) * binom(k1 - p, p1 - p//2)) ** 2
                    else:
                        reps = binom(p, p//2) * binom(k1 - p, p1 - p//2)*binom(k1 - p+1, p1 - p // 2)
                        if p-1 > p//2:
                            reps*=(binom(p-1, p // 2))

                    if enable_tmto==1:
                        start= int(log2(L1))-5
                        end = start + 10
                    else:
                        start=int(ceil(log2(reps)))
                        end=start + 1
                        
                    for l1 in range(start,end):
                        if l1 > l:
                            continue
                            
                        L12 = max(1, L1 ** 2 // 2 ** l1) 

                        qc_advantage=0
                        if qc:
                            L12b = max(1,L1*L1b//2**l1)
                            qc_advantage=log2(k)

                        tmp_mem = log2((2 * L1 + L12) + _mem_matrix(n, k, r)) if not(qc) else log2(L1+L1b + min(L12,L12b) + _mem_matrix(n, k, r))
                        if tmp_mem > mem:
                            continue
                            
                        Tp = max(log2(binom(n, w))
                                 - log2(binom(n - k - l, w - 2 * p + qc))
                                 - log2(binom(k1, p))
                                 - log2(binom(k1, p - qc))
                                 - qc_advantage - solutions, 0)

                        Tg = _gaussian_elimination_complexity(n, k, r)
                        if not(qc):
                            T_tree = 2 * _list_merge_complexity(L1, l1, hmap) + _list_merge_complexity(L12,
                                                                                                   l - l1,
                                                                                                   hmap)
                        else:
                            T_tree =  _list_merge_async_complexity(L1,L1b,l1,hmap)+ _list_merge_complexity(L1,l1, hmap) + _list_merge_async_complexity(L12,L12b,
                                                                                                             l-l1, hmap)



                        T_rep = int(ceil(2 ** (l1 - log2(reps))))

                        tmp = Tp + log2(Tg + T_rep * T_tree)
                        # print(tmp, Tp, T_rep, T_tree)
                        
                        if memory_access==1:
                            tmp += log2(tmp_mem)
                        elif memory_access==2:
                            tmp += tmp_mem/3
                        elif callable(memory_access):
                            tmp += memory_access(tmp_mem)
                            
                        if tmp < time or (tmp==time and tmp_mem<memory):
                            time = tmp
                            memory = tmp_mem
                            params = [p, l, p1,T_tree,Tp,l1,log2(reps)]
                            tree_detail=[log2(Tg), log2(2 * _list_merge_complexity(L1, l1, hmap)),log2(_list_merge_complexity(L12,l - l1,hmap))]
                            lists=[log2(L1),log2(L12),2*log2(L12)-(l-l1)]

        for i in range(len(i_val)):
            if params[i] == i_val[i] - 1:
                stop = False
                i_val[i] += i_val_inc[i]

        if stop == True:
            break
    par = {"l": params[1], "p": params[0], "p1": params[2], "l1": params[5],"reps": params[6], "depth": 2}
    res = {"time": time, "memory": memory, "parameters": par, "perms":params[4],"lists":lists}
    return res


In [15]:
#AES encryptions: kB/s
AES={"128":{"128":16892817.80 ,"192": 14375882.78,"256": 12473219.60 },"256":{"128": 17863577.85,"192":15071777.05,"256":12982587.16 }}

### Compute security of McEliece in comparison to AES

In [16]:
McEliece_level1={"name": "McEliece C1", "n": 3488, "k":2720, "w" : 64}
McEliece_level3={"name": "McEliece C3","n": 4608, "k":3360, "w" : 96}
McEliece_level5a={"name": "McEliece C5a","n": 6688, "k":5024, "w" : 128}
McEliece_level5b={"name": "McEliece C5b","n": 6960, "k":5413, "w" : 119}
McEliece_level5c={"name": "McEliece C5c","n": 8192, "k":6528, "w" : 128}

BIKE_level1={"name": "BIKE C1", "n": 24646, "k":12323, "w": 134 , "w_k" : 142}
BIKE_level3={"name": "BIKE C3","n": 49318, "k":24659, "w": 199 , "w_k" : 206}
BIKE_level5={"name": "BIKE C5","n": 81946, "k":40973, "w": 264 , "w_k" : 274}

HQC_level1={"name": "HQC C1","n": 35338, "k":17669, "w" : 132, "w_e": 75}
HQC_level3={"name": "HQC C3","n": 71702, "k":35851, "w" : 200, "w_e": 114}
HQC_level5={"name": "HQC C5","n": 115274, "k":57637, "w" : 262, "w_e": 149}

In [17]:
verbose=1
levels_mceliece=[[],[],[]]
for access_cost in range(3):
    for mem_indicator in range(3):
        levels_mceliece[access_cost].append([])
        for sec_level in [128,192,256,257,258]:
            switch=sec_level
            if sec_level>256:
                sec_level=256

            AES_blockwidth=128 if sec_level==128 else 256
            AES_kilobytes=AES[str(AES_blockwidth)][str(sec_level)]
            if switch==128:
                c=McEliece_level1
            elif switch==192:
                c=McEliece_level3
            elif switch==256:
                c=McEliece_level5a
            elif switch==257:
                c=McEliece_level5b
            else:
                c=McEliece_level5c
            n=c["n"]
            k=c["k"]-1
            w=c["w"]

            AES_kilobytes*=2

            AES_encryptions_sec=AES_kilobytes*1024*8/AES_blockwidth
            
            AES_encryptions_year=AES_encryptions_sec*60*60*24*365
            AES_years=sec_level-log2(AES_encryptions_year)
            
            McEliece_1284_years=16.01/365
            if mem_indicator==0:
                max_mem=inf
            elif mem_indicator==1:
                max_mem=80-log2(n)
            else:
                max_mem=60-log2(n)

            McEliece_1284_complexity=bjmm_depth_2_qc_complexity(1284,1027,24,memory_access=access_cost,enable_tmto=1)["time"]+log2(1284)
            McEliece_big_complexity=bjmm_depth_2_qc_complexity(n,k,w,memory_access=access_cost,mem=max_mem,enable_tmto=1)["time"]+log2(n)
    
            McEliece_big_years=log2(McEliece_1284_years)+McEliece_big_complexity-McEliece_1284_complexity
        
            if verbose:
                if AES_years-McEliece_big_years<0:
                    print("McEliece is",abs(AES_years-McEliece_big_years),"bits harder")
                else:
                    print("AES is",abs(AES_years-McEliece_big_years),"bits harder")
            levels_mceliece[access_cost][mem_indicator].append(McEliece_big_years-AES_years)

AES is 0.821345382473524 bits harder
AES is 26.100739959886596 bits harder
AES is 24.038515207446324 bits harder
AES is 24.728306937585387 bits harder
McEliece is 5.145061344599071 bits harder
McEliece is 0.29281533306114227 bits harder
AES is 23.945261649805985 bits harder
AES is 13.644594017803996 bits harder
AES is 13.357463334803725 bits harder
McEliece is 21.1764312760493 bits harder
McEliece is 2.6266262081559972 bits harder
AES is 19.84528463662329 bits harder
AES is 9.066757990253336 bits harder
AES is 8.582181528025444 bits harder
McEliece is 26.426355584029352 bits harder
McEliece is 0.8454800983782178 bits harder
AES is 24.152176769490467 bits harder
AES is 21.594866076615148 bits harder
AES is 22.26350210049597 bits harder
McEliece is 7.850812691571548 bits harder
McEliece is 1.543950070934173 bits harder
AES is 22.64732723173941 bits harder
AES is 12.400360902357193 bits harder
AES is 12.106813182807514 bits harder
McEliece is 22.474944190196595 bits harder
McEliece is 3.4

### Computing security of HQC / BIKE in comparison to AES

In [19]:
BIKE_level1={"name": "BIKE C1", "n": 24646, "k":12323, "w": 134 , "w_k" : 142}
BIKE_level3={"name": "BIKE C3","n": 49318, "k":24659, "w": 199 , "w_k" : 206}
BIKE_level5={"name": "BIKE C5","n": 81946, "k":40973, "w": 264 , "w_k" : 274}

HQC_level1={"name": "HQC C1","n": 35338, "k":17669, "w" : 132, "w_e": 75}
HQC_level3={"name": "HQC C3","n": 71702, "k":35851, "w" : 200, "w_e": 114}
HQC_level5={"name": "HQC C5","n": 115274, "k":57637, "w" : 262, "w_e": 149}
params_qc={"BIKEmsg": [BIKE_level1,BIKE_level3,BIKE_level5], "BIKEkey": [BIKE_level1,BIKE_level3,BIKE_level5], "HQC":[HQC_level1,HQC_level3,HQC_level5]}

In [ ]:
verbose=1
levels_qc={"BIKEmsg": [[],[],[]], "BIKEkey": [[],[],[]], "HQC":[[],[],[]]}
for bike_o_hqc in [ "HQC","BIKEmsg", "BIKEkey",]:
    for access_cost in range(3):
        for mem_indicator in range(1):
            levels_qc[bike_o_hqc][access_cost].append([])
            for sec_level in [128,192,256]:
                AES_blockwidth=128 if sec_level==128 else 256
                AES_kilobytes=AES[str(AES_blockwidth)][str(sec_level)]
                if sec_level==128:
                    c= params_qc[bike_o_hqc][0]
                elif sec_level==192:
                    c= params_qc[bike_o_hqc][1]
                else:
                    c= params_qc[bike_o_hqc][2]
                    
                n=c["n"]
                k=c["k"]
                if bike_o_hqc=="BIKEkey":
                    w=c["w_k"]
                else:
                    w=c["w"]

                AES_kilobytes*=2

                AES_encryptions_sec=AES_kilobytes*1024*8/AES_blockwidth
                AES_encryptions_year=AES_encryptions_sec*60*60*24*365
                AES_years=sec_level-log2(AES_encryptions_year)
                
                if mem_indicator==0:
                    max_mem=inf
                elif mem_indicator==1:
                    max_mem=60-log2(n)
                else:
                    max_mem==80-log2(n)

                Experiment_years=38.16/24/365
                Experiment_complexity=bjmm_depth_2_qc_complexity(3138,3138//2,56,qc=1,memory_access=access_cost,enable_tmto=1)["time"]+log2(3138)
                big_complexity=bjmm_depth_2_qc_complexity(n,k,w,mem=max_mem,qc= 0 if bike_o_hqc=="BIKEkey" else 1,memory_access=access_cost,enable_tmto=1)["time"]+log2(n)
                if bike_o_hqc=="BIKEkey":
                    big_complexity-=log2(k)
                big_years=log2(Experiment_years)+big_complexity-Experiment_complexity
                if verbose:
                    if AES_years-big_years<0:
                        #print(abs(AES_years-big_years))
                        name=bike_o_hqc
                        print(name,"is",abs(AES_years-big_years),"bits harder")
                    else:
                        print("AES is",abs(AES_years-big_years),"bits harder")
                levels_qc[bike_o_hqc][access_cost][mem_indicator].append(big_years-AES_years)

AES is 1.840681551672489 bits harder
HQC is 1.192371928521311 bits harder


###  Comparison to gate count

In [36]:
AES_GATE_COUNT_LEVEL={128: 143, 192: 207, 256: 272}

In [39]:
verbose = 1
tm = 0
levels_mceliece = [[],[],[]]
for access_cost in range(3):
    for mem_indicator in range(3):
        levels_mceliece[access_cost].append([])
        for sec_level in [128,192,256,257,258]:
            switch=sec_level
            if sec_level>256:
                sec_level=256

            #AES_blockwidth=128 if sec_level==128 else 256
            #AES_kilobytes=AES[str(AES_blockwidth)][str(sec_level)]
            if switch==128:
                c=McEliece_level1
            elif switch==192:
                c=McEliece_level3
            elif switch==256:
                c=McEliece_level5a
            elif switch==257:
                c=McEliece_level5b
            else:
                c=McEliece_level5c
                
            n=c["n"]
            k=c["k"]-1
            w=c["w"]

            #AES_kilobytes*=2
            #AES_encryptions_sec=AES_kilobytes*1024*8/AES_blockwidth
            #AES_encryptions_year=AES_encryptions_sec*60*60*24*365
            #AES_years=sec_level-log2(AES_encryptions_year)
            AES_years=AES_GATE_COUNT_LEVEL[sec_level]
            #McEliece_1284_years=16.01/365
            
            if mem_indicator==0:
                max_mem=inf
            elif mem_indicator==1:
                max_mem=80-log2(n)
            else:
                max_mem=60-log2(n)

            #McEliece_1284_complexity=bjmm_depth_2_qc_complexity(1284,1027,24,memory_access=access_cost,enable_tmto=1)["time"]+log2(1284)
            McEliece_big_complexity=bjmm_depth_2_qc_complexity(n,k,w,memory_access=access_cost,mem=max_mem,enable_tmto=tm)["time"]+log2(n)
            #McEliece_big_years=log2(McEliece_1284_years)+McEliece_big_complexity-McEliece_1284_complexity
            
            McEliece_big_years=McEliece_big_complexity

            if verbose:
                if AES_years-McEliece_big_years<0:
                    print("McEliece is",abs(AES_years-McEliece_big_years),"bits harder")
                else:
                    print("AES is",abs(AES_years-McEliece_big_years),"bits harder")
                    
            levels_mceliece[access_cost][mem_indicator].append(McEliece_big_years-AES_years)

AES is 0.7540274542168959 bits harder
AES is 24.540525590856305 bits harder
AES is 23.645376766485953 bits harder
AES is 24.256622947152465 bits harder
McEliece is 5.642313802095032 bits harder
McEliece is 0.7007594443273888 bits harder
AES is 21.19977682998109 bits harder
AES is 12.134143588533107 bits harder
AES is 11.330681544355343 bits harder
McEliece is 22.90810966021826 bits harder
McEliece is 3.9579885882590418 bits harder
AES is 18.79705293985441 bits harder
AES is 4.3225077679104515 bits harder
AES is 4.257413402172347 bits harder
McEliece is 32.24255891352749 bits harder
McEliece is 5.658240946533397 bits harder
AES is 18.058570427638813 bits harder
AES is 16.44040936038064 bits harder
AES is 17.02944525348974 bits harder
McEliece is 13.099190093521486 bits harder
McEliece is 6.7465672480876435 bits harder
AES is 15.364603534571813 bits harder
AES is 6.194539946279292 bits harder
AES is 5.364894563045311 bits harder
McEliece is 28.903058058962927 bits harder
McEliece is 9.43

In [40]:
levels_mceliece_notm = levels_mceliece
levels_mceliece_notm

[[[-0.7540274542168959,
   -24.540525590856305,
   -23.645376766485953,
   -24.256622947152465,
   5.642313802095032],
  [0.7007594443273888,
   -21.19977682998109,
   -12.134143588533107,
   -11.330681544355343,
   22.90810966021826],
  [3.9579885882590418,
   -18.79705293985441,
   -4.3225077679104515,
   -4.257413402172347,
   32.24255891352749]],
 [[5.658240946533397,
   -18.058570427638813,
   -16.44040936038064,
   -17.02944525348974,
   13.099190093521486],
  [6.7465672480876435,
   -15.364603534571813,
   -6.194539946279292,
   -5.364894563045311,
   28.903058058962927],
  [9.43119352872543,
   -13.279489514082371,
   0.8050806032297828,
   0.8608862752966502,
   37.419892566164265]],
 [[15.93317455073796,
   -5.545039032779044,
   6.760070185351651,
   7.320771342195201,
   44.16687488573291],
  [15.93317455073796,
   -5.545039032779044,
   6.760070185351651,
   7.320771342195201,
   44.16687488573291],
  [15.93317455073796,
   -5.545039032779044,
   6.760070185351651,
   7.32

In [38]:
levels_mceliece_tm = levels_mceliece
levels_mceliece_tm

[[[-0.9779495076007549,
   -25.092783617692362,
   -23.815287247315013,
   -24.505078977454076,
   5.368289304730354],
  [0.13621120793391128,
   -22.93730530761175,
   -13.421366057672685,
   -13.134235374672414,
   21.399659236180582],
  [2.4700220830287662,
   -18.837328294429057,
   -8.843530030122054,
   -8.358953567894162,
   26.649583544160635]],
 [[5.444506692915439,
   -18.38858970763178,
   -16.616007396819384,
   -17.284643420700206,
   12.829671371367283],
  [6.142976665471394,
   -16.883740169880724,
   -7.4215022225614575,
   -7.127954503011779,
   27.45380286999233],
  [8.061060153216488,
   -13.413648419585797,
   -3.336659005547517,
   -2.826190481144806,
   32.18884197718393]],
 [[14.25051845847264,
   -7.313707668474763,
   4.1252185254462574,
   4.9694468464704755,
   41.118013114818666],
  [14.25051845847264,
   -7.313707668474763,
   4.1252185254462574,
   4.9694468464704755,
   41.118013114818666],
  [14.25051845847264,
   -7.313707668474763,
   4.125218525446257

In [35]:
# data from the `McEliece needs a Break`
levels_mceliece_old = [[
    [0.09, -24.86, -23.18, -23.80, 6.10],
    [1.54, -21.52, -11.67, -10.87, 23.37],
    [4.80, -19.12,  -3.86,  -3.80, 32.70]],
    
    [[1.77, -23.11, -20.70, -21.29, 8.84],
    [2.86, -20.41,  -10.46,  -9.63, 24.64],
    [5.55, -18.33,   -3.46,  -3.40, 33.16]],
    
    [[10.37, -12.27, 0.82, 1.38, 38.22],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
]

# from our new paper
levels_mceliece_old2= [
    [[-0.65, -25.93, -23.86, -24.55, 5,32],
     [0.47, -23.77, -13.47, -13.18, 21.35],
     [2.80, -19.67,  -8.89, -8.41, 26.60]],
    
    [[0.84, -24.16, -21.60, -22.27,  8.84],
     [1.53, -22.66, -12.41, -12.12, 22.47],
     [3.45, -19.19,  -8.32,  -7.81, 27.20]],
    
    [[8.92, -13.81,  -1.59,  -0.75, 35.40],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
]

In [41]:
def padding(i,b=0):
    s=""
    if i<=-10:
        s+="$"+f'{round(i,2):.2f}'+"$"
    elif i<0:
        s+="$-\phantom{}"+f'{abs(round(i,2)):.2f}'+"$" if not(b) else "$-"+f'{abs(round(i,2)):.2f}'+"$"
    elif i<10:
        s+="$\phantom{-}"+f'{abs(round(i,2)):.2f}'+"$" if not(b) else "$\phantom{-}"+f'{abs(round(i,2)):.2f}'+"$"
    else:
        s+="$\phantom{-}"+f'{abs(round(i,2)):.2f}'+"$"

    return s

<>:6: DeprecationWarning: invalid escape sequence '\p'
<>:8: DeprecationWarning: invalid escape sequence '\p'
<>:8: DeprecationWarning: invalid escape sequence '\p'
<>:10: DeprecationWarning: invalid escape sequence '\p'
<>:6: DeprecationWarning: invalid escape sequence '\p'
<>:8: DeprecationWarning: invalid escape sequence '\p'
<>:8: DeprecationWarning: invalid escape sequence '\p'
<>:10: DeprecationWarning: invalid escape sequence '\p'
/tmp/ipykernel_349228/835716141.py:6: DeprecationWarning: invalid escape sequence '\p'
  s+="$-\phantom{}"+f'{abs(round(i,Integer(2))):.2f}'+"$" if not(b) else "$-"+f'{abs(round(i,Integer(2))):.2f}'+"$"
/tmp/ipykernel_349228/835716141.py:8: DeprecationWarning: invalid escape sequence '\p'
  s+="$\phantom{-}"+f'{abs(round(i,Integer(2))):.2f}'+"$" if not(b) else "$\phantom{-}"+f'{abs(round(i,Integer(2))):.2f}'+"$"
/tmp/ipykernel_349228/835716141.py:8: DeprecationWarning: invalid escape sequence '\p'
  s+="$\phantom{-}"+f'{abs(round(i,Integer(2))):.2f}'+"

In [42]:
levels_mceliece_old = levels_mceliece_notm
levels_mceliece = levels_mceliece_tm

for memory_access in range(3):
    # old approach
    # col_st = ["&\tunlimited", "McEliece\t&\t$M\\leq 80$", "\t&\t$M\\leq 60$"]
    col_st = ["unlimited",  "$M\\leq 80$", "$M\\leq 60$"]
    
    #part_st = ["\underline{constant:}\\[0.05cm]", "\underline{logarithmic:}\\[0.05cm]", "\underline{cube-root:}\\[0.05cm]"]
    #test = ["\underline{constant:\\\[0.05cm\]"]
    count = 0
    count2 = 0
    for a,i in enumerate(levels_mceliece[memory_access]):
        #print(part_st[a])
        b = 1
        count3=0
        for j in i:
            l = levels_mceliece_old[memory_access][count2][count3]
            #l = levels_mceliece[memory_access][count2][count3]
            
            # old approach: printing in the beginning the name of the algorithm
            if b:
                print(col_st[count] + " ", end="")
                
            #print(round(l, 3), round(j, 3), f' ({round(l,2)-round(j,2):.2f})', end = "")
            print("\t&\t", padding(j, b), f' ({round(l, 2) - round(j, 2):.2f})',end="")
            
            count3 += 1
            b = 0

        count +=1
        count2 += 1
        print("\\\\[0.1cm]")

unlimited 	&	 $-0.98$  (0.23)	&	 $-25.09$  (0.55)	&	 $-23.82$  (0.17)	&	 $-24.51$  (0.25)	&	 $\phantom{-}5.37$  (0.27)\\[0.1cm]
$M\leq 80$ 	&	 $\phantom{-}0.14$  (0.56)	&	 $-22.94$  (1.74)	&	 $-13.42$  (1.29)	&	 $-13.13$  (1.80)	&	 $\phantom{-}21.40$  (1.51)\\[0.1cm]
$M\leq 60$ 	&	 $\phantom{-}2.47$  (1.49)	&	 $-18.84$  (0.04)	&	 $-\phantom{}8.84$  (4.52)	&	 $-\phantom{}8.36$  (4.10)	&	 $\phantom{-}26.65$  (5.59)\\[0.1cm]
unlimited 	&	 $\phantom{-}5.44$  (0.22)	&	 $-18.39$  (0.33)	&	 $-16.62$  (0.18)	&	 $-17.28$  (0.25)	&	 $\phantom{-}12.83$  (0.27)\\[0.1cm]
$M\leq 80$ 	&	 $\phantom{-}6.14$  (0.61)	&	 $-16.88$  (1.52)	&	 $-\phantom{}7.42$  (1.23)	&	 $-\phantom{}7.13$  (1.77)	&	 $\phantom{-}27.45$  (1.45)\\[0.1cm]
$M\leq 60$ 	&	 $\phantom{-}8.06$  (1.37)	&	 $-13.41$  (0.13)	&	 $-\phantom{}3.34$  (4.15)	&	 $-\phantom{}2.83$  (3.69)	&	 $\phantom{-}32.19$  (5.23)\\[0.1cm]
unlimited 	&	 $\phantom{-}14.25$  (1.68)	&	 $-\phantom{}7.31$  (1.76)	&	 $\phantom{-}4.13$  (2.63)	&	 $\phantom{-}4.97$

### BIKE/HQC Gate count comparison

In [12]:
verbose=1
levels_qc={"BIKEmsg": [[],[],[]], "BIKEkey": [[],[],[]], "HQC":[[],[],[]]}
for bike_o_hqc in [ "HQC","BIKEmsg", "BIKEkey",]:
    for access_cost in range(3):
        for mem_indicator in range(1):
            levels_qc[bike_o_hqc][access_cost].append([])
            for sec_level in [128,192,256]:
                if sec_level==128:
                    c= params_qc[bike_o_hqc][0]
                elif sec_level==192:
                    c= params_qc[bike_o_hqc][1]
                else:
                    c= params_qc[bike_o_hqc][2]
                    
                n=c["n"]
                k=c["k"]
                
                if bike_o_hqc=="BIKEkey":
                    w=c["w_k"]
                else:
                    w=c["w"]
                
                print(n, k, w)
                #AES_kilobytes*=2
                #AES_blockwidth=128 if sec_level==128 else 256
                #AES_kilobytes=AES[str(AES_blockwidth)][str(sec_level)]
                #AES_encryptions_sec=AES_kilobytes*1024*8/AES_blockwidth
                #AES_encryptions_year=AES_encryptions_sec*60*60*24*365
                #AES_years=sec_level-log2(AES_encryptions_year)
                AES_years = AES_GATE_COUNT_LEVEL[sec_level]
                
                if mem_indicator==0:
                    max_mem=inf
                elif mem_indicator==1:
                    max_mem=60-log2(n)
                else:
                    max_mem==80-log2(n)

                #Experiment_years=38.16/24/365
                #Experiment_complexity=bjmm_depth_2_qc_complexity(3138,3138//2,56,qc=1,memory_access=access_cost,enable_tmto=1)["time"]+log2(3138)
                qc = 0 if bike_o_hqc=="BIKEkey" else 1
                
                big_complexity=bjmm_depth_2_qc_complexity         (n,k,w,memory_access=access_cost,mem=max_mem,qc=qc,enable_tmto=1)["time"]+log2(n)
                #McEliece_big_complexity=bjmm_depth_2_qc_complexity(n,k,w,memory_access=access_cost,mem=max_mem,enable_tmto=1)["time"]+log2(n)

                if bike_o_hqc=="BIKEkey":
                    big_complexity-=log2(k)
                
                # big_years = log2(Experiment_years)+big_complexity-Experiment_complexity
                big_years=big_complexity
                #print(big_complexity, AES_years)
                if verbose:
                    if AES_years-big_years<0:
                        name=bike_o_hqc
                        print(name,"is",abs(AES_years-big_years),"bits harder")
                    else:
                        print("AES is",abs(AES_years-big_years),"bits harder")
                        
                levels_qc[bike_o_hqc][access_cost][mem_indicator].append(big_years-AES_years)

35338 17669 132
144.73596073351837 147
AES is 2.264039266481632 bits harder
71702 35851 200
212.93357468103363 207
HQC is 5.933574681033633 bits harder
115274 57637 262
275.09666650413186 272
HQC is 3.0966665041318606 bits harder
35338 17669 132
148.89302457543937 147
HQC is 1.8930245754393695 bits harder
71702 35851 200
217.17643593779644 207
HQC is 10.176435937796441 bits harder
115274 57637 262
279.40152045777074 272
HQC is 7.401520457770744 bits harder
35338 17669 132
150.36654986145675 147
HQC is 3.366549861456747 bits harder
71702 35851 200
218.90769912910815 207
HQC is 11.907699129108153 bits harder
115274 57637 262
281.31564399111056 272
HQC is 9.315643991110562 bits harder
24646 12323 134
145.92339741556088 147
AES is 1.0766025844391152 bits harder
49318 24659 199
211.1525133770194 207
BIKEmsg is 4.152513377019403 bits harder
81946 40973 264
276.2211045313819 272
BIKEmsg is 4.221104531381911 bits harder
24646 12323 134
150.04160109375513 147
BIKEmsg is 3.0416010937551334 bits 

In [16]:
levels_qc

{'BIKEmsg': [[[-1.0766025844391152, 4.152513377019403, 4.221104531381911]],
  [[3.0416010937551334, 8.356007899685665, 8.61776663952412]],
  [[4.398543177607479, 9.96698290920682, 10.406052951525055]]],
 'BIKEkey': [[[0.3085315013121317, 3.6757762557280103, 6.084816065745144]],
  [[4.647138986070928, 8.463212464477238, 11.286188252604688]],
  [[5.91943586576275, 9.992277476159359, 13.009218211411621]]],
 'HQC': [[[-2.264039266481632, 5.933574681033633, 3.0966665041318606]],
  [[1.8930245754393695, 10.176435937796441, 7.401520457770744]],
  [[3.366549861456747, 11.907699129108153, 9.315643991110562]]]}

In [25]:
# from the `McEliece needs a break paper`
levels_qc_old2 = { "BIKEmsg": 
    [[[2.44, 2.50, 3.49]],
     [[3.88, 2.13, 5.87]],
     [[1.24, 4.28, 2.23]]],
    
    "BIKEkey": [[[2.86, 3.04, 4.10],
     [4.42, 3.11, 6.74],
     [1.72, 4.87, 2.90]]],
    
    "HQC": [[[4.47, 5.20, 6.68],
     [5.77, 5.00, 9.03],
     [3.62, 7.34, 5.75]]]
    }

# numbers from the first table of the new paper
levels_qc_old = {
    "BIKEmsg": [[[-0.65, -0.59,  0.26]],
     [[0.73,  -1.73,  2.13]],
     [[-1.84,  1.19, -0.86]]],
    
    "BIKEkey": [[[-0.36, -0.21, -0.84]],
     [[ 1.24, -0.10,  3.5]],
     [[-1.51,  1.61, -0.38]]],
    
    "HQC": [[[0.37, 0.77, 1.99]],
     [[1.89, 0.79, 4,6]],
     [[-0.67, 2.71, -0.9]]]
                }

In [27]:
for memory_access in range(3):
    # old approach
    # col_st = ["&\tunlimited", "McEliece\t&\t$M\\leq 80$", "\t&\t$M\\leq 60$"]
    col_st = ["unlimited",  "$M\\leq 80$", "$M\\leq 60$"]
    
    b=1
    count=0
    for j in levels_qc["BIKEmsg"][memory_access][0]:
        l=levels_qc_old["BIKEmsg"][memory_access][0][count]
        if b:
            print("\multirow{2}{*}{BIKE}\t&\tmessage",end="")
            
        print("\t&\t",padding(j,b), f' ({round(l,2)-round(j,2):.2f})',end="")
        b=0
        count+=1
    print("\\\\[0.1cm]")

    b=1
    count=0
    for j in levels_qc["BIKEkey"][memory_access][0]:
        l=levels_qc_old["BIKEkey"][memory_access][0][count]
        if b:
            print("&\tkey",end="")
            
        print("\t&\t",padding(j,b), f' ({round(l,2)-round(j,2):.2f})',end="")
        b=0
        count+=1
    print("\\\\[0.1cm]")


    b=1
    count=0
    for j in levels_qc["HQC"][memory_access][0]:
        l=levels_qc_old["HQC"][memory_access][0][count]
        if b:
            print("\tHQC\t&",end="")
            
        print("	\t&\t",padding(j,b), f' ({round(l,2)-round(j,2):.2f})',end="")
        b=0
        count+=1
    print("\\\\")

        


\multirow{2}{*}{BIKE}	&	message	&	 $-1.08$  (0.43)	&	 $\phantom{-}4.15$  (-4.74)	&	 $\phantom{-}4.22$  (-3.96)\\[0.1cm]
&	key	&	 $\phantom{-}0.31$  (-0.67)	&	 $\phantom{-}3.68$  (-3.89)	&	 $\phantom{-}6.08$  (-6.92)\\[0.1cm]
	HQC	&		&	 $-2.26$  (2.63)		&	 $\phantom{-}5.93$  (-5.16)		&	 $\phantom{-}3.10$  (-1.11)\\
\multirow{2}{*}{BIKE}	&	message	&	 $\phantom{-}3.04$  (-2.31)	&	 $\phantom{-}8.36$  (-10.09)	&	 $\phantom{-}8.62$  (-6.49)\\[0.1cm]
&	key	&	 $\phantom{-}4.65$  (-3.41)	&	 $\phantom{-}8.46$  (-8.56)	&	 $\phantom{-}11.29$  (-7.79)\\[0.1cm]
	HQC	&		&	 $\phantom{-}1.89$  (0.00)		&	 $\phantom{-}10.18$  (-9.39)		&	 $\phantom{-}7.40$  (-3.40)\\
\multirow{2}{*}{BIKE}	&	message	&	 $\phantom{-}4.40$  (-6.24)	&	 $\phantom{-}9.97$  (-8.78)	&	 $\phantom{-}10.41$  (-11.27)\\[0.1cm]
&	key	&	 $\phantom{-}5.92$  (-7.43)	&	 $\phantom{-}9.99$  (-8.38)	&	 $\phantom{-}13.01$  (-13.39)\\[0.1cm]
	HQC	&		&	 $\phantom{-}3.37$  (-4.04)		&	 $\phantom{-}11.91$  (-9.20)		&	 $\phantom{-}9.32$  (-10.22)\\


<>:11: DeprecationWarning: invalid escape sequence '\m'
<>:11: DeprecationWarning: invalid escape sequence '\m'
/tmp/ipykernel_1942756/1406336278.py:11: DeprecationWarning: invalid escape sequence '\m'
  print("\multirow{2}{*}{BIKE}\t&\tmessage",end="")
